<center><img alt="" src="images/0_Cover.jpg"/></center> 

## <center><font color="blue">Text Analytics</font></center>

<h2 id="(C)-Taufik-Sutanto---2019" style="text-align: center;">(C) Taufik Sutanto - 2019</h2>
<h2 id="tau-data-Indonesia-~-https://tau-data.id" style="text-align: center;">tau-data Indonesia ~ <a href="https://tau-data.id" target="_blank"><span style="color: #0009ff;">https://tau-data.id</span></a></h2>

# Notes and Disclaimer

* This notebook is part of the free (open knowledge) eLearning course at: https://tau-data.id/courses/
* Some images are taken from several resources, we respect those images ownerships and put a reference/citation from where it is originated. Nevertheless, sometimes we are having trouble to find the origin of the image(s). If you are the owner of the image and would like the image taken-out (or want the citation to be revised) from this open knowledge course resources please contact us here with the details: https://tau-data.id/contact/  
* Unless stated otherwise, in general tau-data permit its resources to be copied and-or modified for non-commercial purposes. With condition proper acknowledgement/citation is given.

# Outline:

* Review EDA
* PreProcessing data Teks
* Text Analytics

In [ ]:
# Installing Modules for Google Colab
!wget https://raw.githubusercontent.com/taufikedys/tau-data/master/taudata.py
!mkdir data
!wget -P data/ https://github.com/taufikedys/tau-data/blob/master/data/all_indo_man_tag_corpus_model.crf.tagger
!wget -P data/ https://raw.githubusercontent.com/taufikedys/tau-data/master/data/slang.dic
!wget -P data/ https://raw.githubusercontent.com/taufikedys/tau-data/master/data/stopwords_en.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/tau-data/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/tau-data/master/data/kata_dasar.txt
!pip install unidecode
!pip install textblob
!pip install sastrawi
!pip install spacy
!pip install python-crfsuite
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm
import nltk
nltk.download('popular')

In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import taudata as tau, itertools, re, spacy, nltk, urllib.request
import time, numpy as np, matplotlib.pyplot as plt, networkx as nx, pandas as pd, seaborn as sns 
from matplotlib.colors import ListedColormap
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob, Word
from nltk.tokenize import TweetTokenizer
from spacy.lang.id import Indonesian
from spacy.lang.en import English
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from unidecode import unidecode
from nltk.tag import CRFTagger
from textblob import Word
from html import unescape
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from gensim.models import Word2Vec, FastText

factory = StopWordRemoverFactory()
ct = CRFTagger()
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')
nlp_id = spacy.blank('id') #Indonesian()  # Language Model
nlp_en = spacy.load("en_core_web_sm") # English()
stemmer = StemmerFactory().create_stemmer()
pyLDAvis.enable_notebook()
sns.set(style="ticks", color_codes=True)
random_state = 170

# Review:

* https://tau-data.id/courses/exploratory-data-analysis-eda/
* https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/

## Text Mining dan NLP?
<p>Natural Language Processing (NLP) - Pemrosesan Bahasa Alami (PBA):&nbsp;</p>

<p>
&quot;<big><em>Sebuah cabang ilmu&nbsp;(AI/Computational Linguistik) yang mempelajari bagaimana&nbsp;bahasa (alami) manusia (terucap/tertulis) dapat dipahami dengan baik oleh komputer dan komputer dapat merespon dengan cara yang serupa ke manusia</em></big>&quot;.</p>

<p><img alt="" src="images/1_jarvis.jpg" style="height: 450px; width: 600px;" /></p>

<p><a href="https://www.turn-on.de/lifestyle/topliste/zehn-film-gadgets-die-wir-uns-im-wahren-leben-wuenschen-4413" target="_blank"><strong>[Image Source]</strong></a></p>


<p><strong>Aplikasi Umum NLP:</strong></p>

<ol>
	<li>Machine Translation (Misal&nbsp;https://translate.google.com/ )</li>
	<li>Information Retrieval (IR)&nbsp;(misal www.google.com, bing, elasticsearch, etc.)</li>
	<li>Man-Machine Interface (misal Siri, cortana, atau Alexa)</li>
</ol>


<p><strong>Apakah Perbedaan antara NLP dan Text Mining (TM)?</strong></p>

<p>TM (terkadang disebut Text Analytics) adalah sebuah pemrosesan teks (biasanya dalam skala besar) untuk menghasilkan (generate) informasi atau insights. Untuk menghasilkan informasi TM menggunakan beberapa metode, termasuk NLP. TM mengolah teks secara eksplisit, sementara NLP mencoba mencari makna latent (tersembunyi) lewat aturan bahasa (e.g. grammar/idioms/Semantics).<br />
<strong>Contoh aplikasi TM</strong> : Clustering, Klasifikasi, Social Media Analytics (SMA).</p>


<p><img alt="https://www.kdnuggets.com/2017/11/framework-approaching-textual-data-tasks.html" src="images/1_NLP_TextMining.jpg" style="height: 470px; width: 600px;" /></p>

<p>[image source: <a href="https://www.elsevier.com/books/practical-text-mining-and-statistical-analysis-for-non-structured-text-data-applications/miner/978-0-12-386979-1" target="_blank">Gary M.:&quot;Practical Text Mining and Statistical Analysis for Non-structured Text Data Applications&quot;</a>]</p>


<p><img alt="" src="images/1_Text_Analytics.jpg" style="height: 451px; width: 600px;" /></p>

<p>[Image Source: <a href="http://www.pearson.com.au/products/S-Z-Turban-Sharda/Business-Intelligence-and-Analytics-Systems-for-Decision-Support-Global-Edition/9781292009209?R=9781292009209" target="_blank">Efraim T. &quot;Business Intelligence and Analytics: Systems for Decision Support, Global Edition (10e)</a>&quot;]</p>


## Tokenisasi

<p>Tokenisasi adalah pemisahan kata, simbol, frase, dan entitas penting lainnya (yang disebut sebagai token) dari sebuah teks untuk kemudian di analisa lebih lanjut. Token dalam NLP sering dimaknai dengan &quot;sebuah kata&quot;, walau tokenisasi juga bisa dilakukan ke kalimat, paragraf, atau entitas penting lainnya (misal suatu pola string DNA di Bioinformatika).</p>

<p><strong>Mengapa perlu tokenisasi?</strong></p>

<ul>
	<li>Langkah penting dalam preprocessing, menghindari kompleksitas mengolah langsung pada string asal.</li>
	<li>Menghindari masalah (semantic) saat pemrosesan model-model natural language.</li>
	<li>Suatu tahapan sistematis dalam merubah unstructured (text) data ke bentuk terstruktur yang lebih mudah di olah.</li>
</ul>

<p><img alt="" src="images\2_Pipeline_Tokenization.png" style="height:300px; width:768px" /><br />
[<a href="https://www.softwareadvice.com/resources/what-is-text-analytics/" target="_blank"><strong>Image Source</strong></a>]</p>


## Tokenisasi dengan <font color="blue"> TextBlob</font>
<strong>Kelebihan</strong>:</p>
<ol>
	<li>Sederhana &amp; mudah untuk digunakan/pelajari.</li>
	<li>Textblob objects punya behaviour/properties yang sama dengan string di Python.</li>
	<li>TextBlob dibangun dari kombinasi modul NLTK dan (Clips) Pattern</li>
</ol>

<p><strong>Kekurangan</strong>:</p>
<ol>
	<li>Tidak secepat Spacy dan NLTK</li>
	<li>Language Model terbatas: English, German, French</li>
</ol>

<p>*Blob : Binary large Object</p>

In [ ]:
# Tokenizing di TextBlob
T = "Hello, Mr. Man. He smiled!! This, i.e. that, is it."
print(TextBlob(T).words)

In [ ]:
kalimatS = TextBlob(T).sentences
print([str(kalimat) for kalimat in kalimatS])

In [ ]:
# Saat melakukan coding di Python, selalu perhatikan "tipe data" yang dihasilkan oleh modul.
A = TextBlob(T).sentences
B = TextBlob(T).words
print(A[0], type(A[0]))
print(B[0], type(B[0]))
# Apakah bedanya dengan tipe data str biasa di python?

In [ ]:
C = B[0] # <class 'textblob.blob.Word'>
D = 'teks' # tipe string biasa di Python

In [ ]:
# "properties" Blob word
print(dir(C))

In [ ]:
# "properties" string di Python
print(dir(D))

## Tokenisasi tidak hanya language dependent, tapi juga environment dependent

<p>Tokenization sebenarnya tidak sesederhana memisahkan berdasarkan spasi dan removing symbol. Sebagai contoh dalam bahasa Jepang/Cina/Arab suatu kata bisa terdiri dari beberapa karakter.</p>

<p><img alt="" src="images/2_Tokenization_Complexity.jpg" style="height:500px; width:686px" /><br />
[<a href="http://aclweb.org/anthology/Y/Y11/Y11-1038.pdf" target="_blank"><strong>Image Source</strong></a>]</p>


In [ ]:
# Contoh Tokenizer untuk twitter

Tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
tweet = "@Kirana_Sutanto seneng bangeeeeeeeeettttttttt"
print(Tokenizer.tokenize(tweet))

# Masih salah (i.e. "happpy"), nanti kita akan perbaiki ini dengan "spell check"
# catatan: pada permasalahan "Sentiment analysis" kata yang ditulis panjang seperti diatas 
# bisa mengindikasikan sentiment yang kuat

## Tokenisasi (NLP) Bahasa Indonesia:

<p>NLTK belum support Bahasa Indonesia, bahkan module NLP Python yang support bahasa Indonesia secara umum masih sangat langka. Beberapa <u><strong>resources </strong></u>yang dapat digunakan:</p>

<ol>
	<li><strong><a href="https://github.com/kirralabs/indonesian-NLP-resources" target="_blank">KirraLabs</a></strong>: Mix of NLP-TextMining resources</li>
	<li><strong><a href="https://pypi.python.org/pypi/Sastrawi/1.0.1" target="_blank">Sastrawi 1.0.1</a>:</strong>&nbsp;untuk &quot;stemming&quot; &amp;&nbsp;<strong><a href="https://devtrik.com/python/stopword-removal-bahasa-indonesia-python-sastrawi/" target="_blank">stopwords&nbsp;</a></strong>bahasa Indonesia.</li>
	<li><strong><a href="http://stop-words-list-bahasa-indonesia.blogspot.co.id/2012/09/daftar-kata-dasar-bahasa-indonesia.html" target="_blank">Daftar Kata Dasar Indonesia</a></strong>:&nbsp;Bisa di load sebagai dictionary di Python</li>
	<li><strong><a href="https://id.wiktionary.org/wiki/Wiktionary:ProyekWiki_bahasa_Indonesia/Daftar_kata" target="_blank">Wiktionary</a></strong>: ProyekWiki bahasa Indonesia [termasuk Lexicon]</li>
	<li><a href="http://wn-msa.sourceforge.net/" target="_blank"><strong>WordNet Bahasa Indonesia</strong></a>: Bisa di load&nbsp;sebagai dictionary (atau NLTK<em>*</em>) di Python.</li>
	<li><strong><a href="http://kakakpintar.com/daftar-kata-baku-dan-tidak-baku-a-z-dalam-bahasa-indonesia/" target="_blank">Daftar Kata Baku-Tidak Baku</a></strong>: Bisa di load sebagai dictionary di Python.</li>
	<li><strong><a href="https://spacy.io/" target="_blank">Spacy</a></strong>: Cepat/efisien, MIT License, tapi language model Indonesia masih terbatas.</li>
	<li><a href="http://ufal.mff.cuni.cz/udpipe" target="_blank"><strong>UdPipe</strong></a>: Online request &amp; restricted license (support berbagai bahasa -&nbsp;pemrograman).</li>
</ol>

In [ ]:
# Contoh Tokenisasi dalam bahasa Indonesia dengan Spacy
teks = 'Sore itu, Hamzah melihat kupu-kupu di taman. Ibu membeli oleh-oleh di pasar'
tokenS_id = nlp_id(teks)
print([t for t in tokenS_id])

In [ ]:
# Jika menggunakan Language model English:
tokenS_en = nlp_en(teks)
print([token.text for token in tokenS_en])

<p><u><big><strong>Word Case</strong></big></u><big> (Huruf BESAR/kecil):</big></p>

<ul>
	<li>Untuk menganalisa makna (<em>semantic</em>) dari suatu (frase) kata dan mencari informasi dalam proses textmining, seringnya (*) kita tidak membutuhkan informasi huruf besar/kecil dari kata&nbsp;tersebut.</li>
	<li><em>Text case normaliation</em> dapat dilakukan pada string secara efisien tanpa melalui tokenisasi (mengapa?).</li>
	<li>Namun, bergantung pada analisa teks yang akan digunakan pengguna harus berhati-hati dengan urutan proses (pipelining) dalam preprocessing. Mengapa dan apa contohnya?</li>
</ul>

<p>(*) Coba temukan minimal 2 pengecualian dimana&nbsp; huruf kapital/kecil (case) mempengaruhi makna/pemrosesan teks.</p>

In [ ]:
# Ignore case (huruf besar/kecil)
T = "Apa Kabar?"
print(T.lower())
print(T.upper())
# Perintah ini sangat efisien karena hanya merubah satu bit di setiap (awal) bytes dari setiap karakter
# Sehingga tetap efisien jika ingin dilakukan sebelum tokenisasi

## Morphological-Linguistic Normalization: Stemming &amp; Lemmatization
(Canonical Representation)
<p><img alt="" src="images/2_yoda.jpg" style="height:400px; width:400px" /></p>

## <font color="blue">Stemming dan Lemma</font>

<ol>
	<li>
	<p><strong>Stemmer</strong>&nbsp;akan menghasilkan sebuah bentuk kata yang disepakati oleh suatu sistem tanpa mengindahkan konteks kalimat. Syaratnya beberapa kata dengan makna serupa hanya perlu dipetakan secara konsisten ke sebuah kata baku.&nbsp;Banyak digunakan di IR &amp;&nbsp;komputasinya relatif sedikit. Biasanya dilakukan dengan menghilangkan imbuhan (suffix/prefix).</p>
	</li>
	<li>
	<p><strong>lemmatisation</strong> akan menghasilkan kata baku (dictionary word) dan bergantung konteks.</p>
	</li>
	<li>
	<p>Lemma &amp; stemming bisa jadi sama-sama menghasilkan suatu akar kata (root word). Misal : <em>Melompat </em>==&gt; <em>lompat</em></p>
	</li>
</ol>

<p><strong>Mengapa melakukan Stemming &amp; Lemmatisasi</strong>?</p>

<ol>
	<li>Sering digunakan di IR (Information Retrieval) agar ketika seseorang mencari kata tertentu, maka seluruh kata yang terkait juga diikutsertakan.<br />
	Misal:&nbsp;<em>organize</em>,&nbsp;<em>organizes</em>, and&nbsp;<em>organizing&nbsp;</em>&nbsp;dan&nbsp;<em>democracy</em>,&nbsp;<em>democratic</em>, and&nbsp;<em>democratization</em>.</li>
	<li>Di Text Mining Stemming dan Lemmatisasi akan mengurangi dimensi (mengurangi variasi morphologi), yang terkadang akan meningkatkan akurasi.</li>
	<li>Tapi di IR efeknya malah berkebalikan: <strong><font color="blue">meningkatkan recall, tapi menurunkan akurasi&nbsp;</font></strong>[<a href="https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html" target="_blank"><strong>Link</strong></a>]. Contoh: kata&nbsp;<em>operate, operating, operates, operation, operative, operatives, dan operational</em>&nbsp;jika di stem menjadi <em>operate</em>, maka ketika seseorang mencari &quot;<em>operating system</em>&quot;, maka entry seperti&nbsp;<em>operational and research</em> dan&nbsp;<em>operative and dentistry</em>&nbsp;akan muncul sebagai entry dengan relevansi yang cukup tinggi.</li>
</ol>


In [ ]:
# Lemmatizer dengan Sastrawi

I = "perayaan itu Berbarengan dengan saat kita bepergian ke Makassar"
print(stemmer.stem(I))
print(stemmer.stem("Perayaan Bepergian Menyuarakan"))
# Ada beberapa hal yang berbeda antara Sastrawi dan modul-modul diatas.
# Apa sajakah?

## Review Tipe Variabel di Python (string)

<ul>
	<li>Review String di Python</li>
	<li>Filtering (stopwords)</li>
	<li>Replace slang/typos/singkatan</li>
    <li>Spell Check</li>
	<li>Machine translation</li>
	<li>Reguler expression</li>
	<li>Encodings</li>
</ul>


In [ ]:
a = 2.3 # Floating Point (tidak sama dengan bilangan Real)
b = 3.0   # Integer
c = True # T/F Boolean
d = 'python' # String
e = [a,b,c,d] # List
f = (a,b,c,d) # Tuple
g = set([a,b,c,d]) # Set
h = {'a':1, 'b':2, 7:'abc'} # Dictionary : keys, values, items

In [ ]:
b = b/2
type(b)

In [ ]:
# Tekan "d." lalu tekan tombol "tab"
#d.

In [ ]:
# Sehingga kita bisa melakukan preprocessing dasar dengan string manipulation seperti berikut:
S = 'coba '
print(S)
print(S.strip())
print(S.replace("o","i"))
print(S.isalnum())
print(S.strip().isalnum()) # dsb

In [ ]:
'237'.isdigit()

In [ ]:
# String adalah Tuple, sehingga
S1 = 'apa '
S2 = 'kabar'
S1+S2

In [ ]:
S2 = 'abcdefghijklmnopqrst'
S2[9:]

In [ ]:
# String adalah Tuple sehingga bisa di akses seperti List
S = 'NLP dan TextMining di Python'
print('7 character pertama: ', S[:7])
print('7 character terakhir: ', S[-7:])

In [ ]:
# Tapi hati-hati karena ia tuple maka:
try:
    S[5] = 'o'
except:
    print('Error, tuple tidak bisa dirubah nilainya (inplace)')

In [ ]:
S.find('dan')

## Text Level Normalization: StopWords
<p><u>Di Text Mining</u> kata-kata yang <strong>sering muncul </strong>(dan jarang sekali muncul) memiliki sedikit sekali informasi (signifikansi) terhadap model (machine learning) yang digunakan. Hal ini di karenakan kata-kata tersebut muncul di semua kategori (di permasalahan klasifikasi) atau di semua cluster (di permasalahan pengelompokan/clustering). Kata-kata yang sering muncul ini biasa disebut &quot;StopWords&quot;. Stopwords berbeda-beda bergantung dari Bahasa dan Environment (aplikasi)-nya.<br />
<strong>Contoh</strong>:<br />

<ul>
	<li>Stopwords bahasa Inggris: am, is, are, do, the, of, etc.</li>
	<li>Stopwords bahasa Indonesia: adalah, dengan, yang, di, ke, dsb</li>
	<li>Stopwords twitter: RT, ...<br />
	<img alt="" src="images/2_StopWords.png" style="height:250px; width:419px" /></li>
</ul>


In [ ]:
# Loading Stopwords: Ada beberapa cara

NLTK_StopWords = stopwords.words('english')
Sastrawi_StopWords_id = factory.get_stop_words()
Personal_StopWords_en = [t.strip() for t in tau.LoadDocuments(file = 'data/stopwords_en.txt')[0]]
Personal_StopWords_id = [t.strip() for t in tau.LoadDocuments(file = 'data/stopwords_id.txt')[0]]

print(NLTK_StopWords[:5])
print(Sastrawi_StopWords_id)
print(Personal_StopWords_en[:5])
print(Personal_StopWords_id[:5])
print(len(Sastrawi_StopWords_id), len(Personal_StopWords_id), len(NLTK_StopWords), len(Personal_StopWords_en))

### Diskusi: Apakah sebaiknya kita menggunakan daftar stopwords bawaan modul atau custom milik kita sendiri?

In [ ]:
# Tips: selalu rubah list stopwords ke bentuk set, karena di Python jauh lebih cepat untuk cek existence di set ketimbang list
NLTK_StopWords = set(NLTK_StopWords)
Sastrawi_StopWords_id = set(Sastrawi_StopWords_id)
Personal_StopWords_en = set(Personal_StopWords_en)
Personal_StopWords_id = set(Personal_StopWords_id)

In [ ]:
St = ['ada', 'pesawat']
'ada' in St

In [ ]:
A = set([2,3,5])
A

In [ ]:
# Cara menggunakan stopwords

T = "I am doing NLP with tau-data,... \
    adapun saya anu sedang belajar NLP bersama tau-data"
T = T.lower()
Sastrawi_StopWords_id.add('sedang')
Tokens = TextBlob(T).words # Tokenisasi 
T2 = [t for t in Tokens if t not in Sastrawi_StopWords_id] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
T2 = [t for t in T2 if t not in Sastrawi_StopWords_id] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
print(' '.join(T2))
# Catatan: Selalu lakukan Stopword filtering setelah tokenisasi (dan normalisasi).

<p><img alt="" src="images/2_Tokenization_Stopwords.png" style="height:400px; width:765px" /></p>

<p>[<a href="http://chdoig.github.io/acm-sigkdd-topic-modeling/#/6/2" target="_blank">image source</a>]</p>


## Menangani Slang atau Singkatan di Data Teks 

In [ ]:
# Sebuah contoh sederhana 
T = 'jangan ragu gan, langsung saja di order pajangannya.'
# Misal kita hendak mengganti setiap singkatan (slang) dengan bentuk penuhnya. 
# Dalam hal ini kita hendak mengganti 'gan' dengan 'juragan'
H = T.replace('gan','juragan')
print(H)
# Kita tidak bisa melakukan ini

In [ ]:
D = {'yg':'yang', 'gan':'juragan'}
D['yg']

In [ ]:
# dengan tokenisasi
slangs = {'gan':'juragan', 'yg':'yang', 'dgn':'dengan'} #dictionary sederhana berisi daftar singkatan dan kepanjangannya

T = 'jangan ragu gan, langsung saja di order pajangan yg diatas.'
T = TextBlob(T).words
T

In [ ]:
for i,t in enumerate(T):
    if t in slangs.keys():
        T[i] = slangs[t]
print(' '.join(T))

In [ ]:
# Loading Slang dan Singkatan dari File
# Contoh memuat word fix melalui import file. 
slangS = tau.LoadDocuments(file = 'data/slang.dic')[0]
slangS = [t.split(":") for t in slangS]
slangS = [[k.strip(), v.strip()] for k,v in slangS]
slangS = {k:v for k,v in slangS} # rubah ke dalam bentuk dictionary
slangS

In [ ]:
# Test it!
tweet = 'I luv u say. serius gan!, tapi ndak tau kalau sesok.'
T = TextBlob(tweet).words

for i,t in enumerate(T):
    if t in slangS.keys():
        T[i] = slangS[t]
        
print(' '.join(T))

## Machine Language Detection and Translation
<p><img alt="" src="images/2_translation.jpg" style="height:400px; width:534px" /></p>


<p>[<a href="https://www.slideshare.net/PhuLeTruongVinh/ltvp-thesis-defense" target="_blank">image Source</a>]</p>

In [ ]:
#Language Detection (TextBlob)
T = "Aku ingin mengerti NLP dalam bahasa Inggris"
U = "opo iso mangan sa iki"
print(TextBlob(T).detect_language())
print(TextBlob(U).detect_language())

In [ ]:
# Machine Translation (TextBlob)
# Butuh koneksi internet, limited calls. Error otherwise. Need "try" and "catch".
T = "Aku ingin mengerti NLP dalam bahasa Inggris. I love you"
print(TextBlob(T).translate(to='en'))
print(TextBlob(T).translate(to='ar-sa'))
print(TextBlob(T).translate(to='ja'))
# daftar kode bahasa : http://www.cardinalpath.com/resources/tools/google-analytics-language-codes/
# Perhatikan TextBlob secara automatis akan mendeteksi bahasa asal

In [ ]:
# Kalau secara spesifik ingin translate dari suatu bahasa ke bahasa lain:
T = "Aku hanya ingin mengatakan : I Love you"
print(TextBlob(T).translate(from_lang ='id', to='en'))
print(TextBlob(T).translate(from_lang ='en', to='id'))

<p><img alt="" src="images/2_regex_meme.jpg" style="height:397px; width:599px" /></p>

<p><strong>Beberapa Reguler Expression yang sering digunakan di NLP/Text Mining</strong></p>

<ol>
	<li>Menghilangkan/extract email</li>
	<li>Menghilangkan/extract nomer telephone</li>
	<li>Menghilangkan/extract URL di string.</li>
	<li>Alpha Numeric filtering</li>
	<li>Wild Card Search</li>
	<li>Cleaning hashTags di Media Sosial</li>
</ol>


In [ ]:
# Extracting atau replacing eMail.
emailPattern = re.compile(r'[\w._%+-]+@[\w\.-]+\.[a-zA-Z]{2,4}')

txt = 'Contact kami di admin@nlpindonesia.org, nlp.indonesia@sci.yahoo.co.id, atau nlp_nusantara@internet.net'

print( re.sub(emailPattern, ' ', txt) )# clean email
eMailS = re.findall( emailPattern, txt )
print( 'email yang ditemukan: ', str(eMailS) )

<p><img alt="" src="images/2_email_regex.gif" style="height:403px; width:687px" /></p>


In [ ]:
# Pola telephone : \d penanda angka di reguler Expression, \s spasi, dan "|" adalah "atau"
# "?" menyatakan pilihan (optional): colou?r sesuai dengan colour atau color.

phonePattern = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
txt = 'Contact kami di 021-7634562 atau 021-763-4562 atau 021 763 4562 atau 0822959020 atau +628199258688'
print(re.sub(phonePattern,'***',txt))# clean phone
    
noTelp = re.findall(phonePattern,txt)
print('Nomer telephone yang ditemukan: ',str(noTelp))

In [ ]:
# Pola telephone 2: untuk setiap angka 8-14 digits dipisahkan oleh "spasi", "," atau "."
phonePattern = re.compile(r'\b\d{8,14}\b')
txt = 'Contact kami di 082295203040 atau +6282295203040'
print(re.sub(phonePattern,'***',txt))# clean phone
    
noTelp = re.findall(phonePattern,txt)

for no in noTelp:
    if no[0]!='0':
        print('+' + no)
    else:
        print(no)
#print('Nomer telephone yang ditemukan: ',str(noTelp))

In [ ]:
# Website URLS http(s) .... untuk ftp trivial
urlPattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

txt = 'website reguler expression & my site : https://www.regular-expressions.info/ & https://tau-data.id'
print(re.sub(urlPattern,' ',txt))# clean urls

URLs = re.findall(urlPattern,txt)# get URLs
print('URL yang ditemukan: ',str(URLs))

In [ ]:
# cleaning non alpha-numeric
txt = 'Hi! @Mukidi, apa kabar? #sapa_Pagi.'
print(re.sub(r'[^\w]',' ',txt))
# atau jika ingin exclude titik dan koma 
# re.sub(r'[^.,a-zA-Z0-9 \n\.]','',txt)

In [ ]:
# alternative 2:
print(''.join([t for t in txt if t.isalnum() or t==' ']))
# ada perbedaan?

<img alt="" src="images/2_nonStandard_Language.jpg" style="height:359px; width:638px" />

In [ ]:
# Cleaning hashTags dalam posting media sosial
tweet = 'oh IoT, #AndaiSajaIaTahu #ApaYangAkuRasah... #AlayersTweet #d2d'

getHashtags = re.compile(r"#(\w+)")
print("Tags = {0}".format(re.findall(getHashtags, tweet)))
# temukan hanya tags ... perhatikan IoT bukan Tags walau ada huruf besar & kecil dalam satu kata

In [ ]:
pisahtags = re.compile(r'[A-Z][^A-Z]*')

for tags in re.findall(getHashtags, tweet):
    print(re.findall(pisahtags, tags))

In [ ]:
# Mengganti hashtags dengan kata dasar pembentuknya
tweet = 'oh IoT, #AndaiSajaIaTahu #ApaYangAkuRasah... #AlayersTweet'
tagS = re.findall(getHashtags, tweet)
for tag in tagS:
    proper_words = ' '.join(re.findall(pisahtags, tag))
    tweet = tweet.replace('#'+tag,proper_words)

print(tweet)

## Encoding-Decoding:

<ul>
	<li>Hal berikutnya yang perlu diperhatikan dalam memproses data teks adalah encoding-decoding.</li>
	<li>Contoh Encoding: ASCII, utf, latin, dsb.</li>
	<li>saya membahas lebih jauh tetang encoding disini:&nbsp;<br />
	<a href="https://taufiksutanto.blogspot.co.id/2018/01/pereda-sakit-kepala-urgensi-memahami.html" target="_blank">https://taufiksutanto.blogspot.co.id/2018/01/pereda-sakit-kepala-urgensi-memahami.html</a></li>
	<li>Berikut adalah sebuah contoh sederhana tantangan proses encoding-decoding ketika kita hendak memproses data yang berasal dari internet atau media sosial.</li>
</ul>


In [ ]:
# kita bisa menggunakan modul unidecode untuk mendapatkan representasi ASCII terdekat
T = "ḊḕḀṙ ₲ØĐ, p̾l̾e̾a̾s̾e ḧḕḶṖ ṁḕ"
print(unidecode(T).lower())
# Bahasa Indonesia dan Inggris secara umum mampu direpresentasikan dalam encoding ASCII: 
# https://en.wikipedia.org/wiki/ASCII

In [ ]:
# Kita juga bisa membersihkan posting media sosial/website dengan entitas html menggunakan fungsi "unescape" di modul "html"

print(unescape('Satu &lt; Tiga&nbsp;&amp; &#169; adalah simbol Copyright'))

# Social Media Analytics (SMA)

<h3 id="SMA-adalah-sebuah-proses-pengumpulan-data-dari-media-sosial-dan-analisanya-untuk-mendapatkan-'insights'-atau-informasi-berharga-untuk-suatu-tujuan-tertentu-(definisi-adopted-dari-Gartner*)">SMA adalah sebuah proses pengumpulan data dari media sosial dan analisanya untuk mendapatkan &#39;insights&#39; atau informasi berharga untuk suatu tujuan tertentu (definisi diadopsi dari Gartner*).</h3>

<p><img alt="" src="images/8_SMA.JPG" style="width: 600px; height: 304px;" /></p>


<h1 id="Crawling-Data">Crawling/Scrapping Data</h1>

<p><img alt="" src="images/Digital_Media_Crawling_.png" /></p>

* Credits, image source: https://www.promptcloud.com/blog/scraping-social-media-data-for-sentiment-analysis/

# Scrapping

## Wajib menggunakan browser FireFox
## https://twitter.com/search-advanced
## Save as html Complete
## Upload file html ke Google Colab

In [ ]:
# Contoh mengubah html hasil scrapping ke CSV
fileSource = 'data.html'
fileOutput = 'data.csv'

tau.twitter_html2csv(fileSource, fileOutput)

<h2>Visualisasi ~ Simple Text Analytics</h2>

<ul>
	<li>Tidak seperti data terstruktur, data tidak terstruktur seperti teks termasuk salah satu data yang cukup sulit untuk divisualisasikan.<br />
	<img alt="" src="images/11_charts.jpg" style="height:150px; width:276px" /></li>
	<li>Namun terdapat Tools seperti Voyant yang dapat membantu dalam visualisasi sekaligus analisis.<br />
	<img alt="" src="images/11_voyant.png" style="height:118px; width:426px" /></li>
    <li><a href="https://voyant-tools.org/" target="_blank">https://voyant-tools.org/</a></li>
</ul>


<h3 id="Penggunaan-Voyant-1:-WordClouds">Penggunaan Voyant 1: WordClouds</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.</li>
	<li>slider terms: mengkontrol banyaknya terms yang disertakan.</li>
	<li><strong>Summary </strong>(statistics)</li>
	<li><strong>Documents </strong>==&gt; add more</li>
	<li><strong>Phrases </strong>(n-grams like)</li>
	<li><strong>Export </strong>Visualisasi (kanan atas - pertama)</li>
	<li><strong>Options </strong>(kanan atas ke-3): Font, size, stopwords, whitelist</li>
	<li>&quot;?&quot; ==&gt; More Help</li>
</ol>

<p>&nbsp;</p>


<h3 id="Penggunaan-Voyant-2:-Bubbles">Penggunaan Voyant 2: Bubbles</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>&nbsp;Klik tanda 4-kotak kecil (kanan atas ke-3)</li>
	<li>Pilih Visualization Tools ==&gt; Bubbles</li>
	<li>Option: hanya stopwords</li>
	<li>Export: Hanya PNG</li>
</ol>

<p>&nbsp;</p>


<h3 id="Penggunaan-Voyant-3:-Word-Tree">Penggunaan Voyant 3: Word Tree</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Klik branch untuk expand</li>
</ol>


<h3 id="Penggunaan-Voyant-2:-Bubbles">Penggunaan Voyant 4: Links</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Visualization Tools ==&gt; Links</li>
	<li>Klik sembarang terms untuk expand</li>
</ol>


<h3 id="Penggunaan-Voyant-5:-Trends">Penggunaan Voyant 5: Trends</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Document Tools ==&gt; Trends</li>
	<li>.. Butuh preprocessing ...&nbsp;</li>
	<li>Data harus terurut waktu</li>
	<li>Berikut contohnya</li>
</ol>


<h1>End of Module</h1>

<hr />
<p><img alt="" src="images/1_Computer_feeling.jpg" style="height: 463px; width: 600px;" /></p>
